In [1]:
#!pip install networkx

In [1]:
import networkx as nx
from collections import defaultdict, Counter
from typing import List, Tuple, Set, Dict

In [74]:
"""Need to be able to do the reverse as well -- go from a dict of general ledger codes to a graph representation.
That part should be pretty easy. It's easy to see from a code who the ancestors are."""

"Need to be able to do the reverse as well -- go from a dict of general ledger codes to a graph representation.\nThat part should be pretty easy. It's easy to see from a code who the ancestors are."

In [2]:
class Inventory:
    def __init__(self, nodes: Set[str], edges: Set[Tuple[str,str]]):
        
        self.validate_edges(edges)
        self.graph = nx.DiGraph()
        self.graph.add_nodes_from(nodes)
        self.graph.add_edges_from(edges)
        self.set_indices()
        self.code_map = self.create_codes()


    def validate_edges(self, edges: Set[Tuple[str,str]]):
        """
        Check that each child has only one parent
        
        TODO use typechecking to enforce that each tuple is a pair of strings
        """
        #Each child has only one parent implies that no node appears more than once as the second element in an edge
        children = [edge[1] for edge in edges]
        if len(children) != len(set(children)):
            raise Exception("Some nodes have more than one parent!")
    
    def set_indices(self):
        seen = Counter()
        indices = dict()
        for e in self.graph.edges:
            seen[e[0]]+=1
            indices[e[1]] = seen[e[0]]

        #set indices of root nodes
        root_nodes = [node for node, degree in self.graph.in_degree() if degree == 0]
        for i, rt in enumerate(root_nodes):
            indices[rt] = i+1
     
        nx.set_node_attributes(self.graph, indices, "index")
        
        
    def get_parent(self, node: str) -> str:
        if not node in self.graph.nodes:
            raise Exception(f"{node} not in nodes: {list(G.nodes)}")
        for edge in self.graph.edges:
            if edge[1] == node:
                return edge[0]
                
    def get_ancestors(self,node, ancestors=None) -> List[str]:
        """Get the ancestors of a node recursively.
        we use the convention that every node is an ancestor of itself"""
        
        ancestors = ancestors or [node]
        parent = self.get_parent(node)
        if not parent:
            return ancestors
        else:
            ancestors.append(parent)
            anc = self.get_ancestors(parent, ancestors)
            return anc

    def get_depth(self,node: str)-> int:
        """Get the number of ancestors of a node (including self)"""
        return len(self.get_ancestors(node))
        
        
    def get_max_depth(self):
        return max([self.get_depth(nd) for nd in self.graph.nodes])
        
        
    def get_code(self,node):
        """Construct the general ledger code of an item in the inventory"""
        
        index = nx.get_node_attributes(self.graph,"index")[node]
        multiplier = 10**(self.get_max_depth()-self.get_depth(node))
        parent = self.get_parent(node)
        if self.get_depth(node) == 1:
            return index*multiplier
        else:
            return index*multiplier + self.get_code(parent)

    def create_codes(self) -> Dict[int,str]:
        code_item = {self.get_code(nd): nd for nd in self.graph.nodes}
        return {k: v for k,v in sorted(code_item.items())}

            

In [3]:
inv = Inventory(nodes=set(['seafood', 'meat', 'fish', 'mussels', 'steak', 'chicken', "snapper", "chicken", "drumstick", "quarter", "filet mignon", "12oz", "10oz", "8oz"]),
               edges =set([('seafood', 'fish'), ('seafood', 'mussels'), ('meat', 'steak'), ('meat', 'chicken'), ('fish', 'snapper'), ('steak', 'filet mignon'), ('chicken', 'quarter'), ('chicken', 'drumstick'), ('filet mignon', '8oz'), ('filet mignon', '10oz'), ('filet mignon', '12oz')]))

In [4]:
inv.code_map

{1000: 'seafood',
 1100: 'mussels',
 1200: 'fish',
 1210: 'snapper',
 2000: 'meat',
 2100: 'steak',
 2110: 'filet mignon',
 2111: '8oz',
 2112: '10oz',
 2113: '12oz',
 2200: 'chicken',
 2210: 'drumstick',
 2220: 'quarter'}

In [23]:
def code_map_to_edges(code_map: Dict[int,str]) -> List[Tuple[str,str]]:
    nodes = list(code_map.keys())
    edges = list()
    for code, nd_name  in code_map.items():
        parent_code = get_code_parent(code)
        parent_name = code_map[parent_code]
        edges.append((parent_name,nd_name))
    return edges
        
        
        


In [25]:
code_map_to_edges(inv.code_map)

[('seafood', 'seafood'),
 ('mussels', 'mussels'),
 ('fish', 'fish'),
 ('snapper', 'snapper'),
 ('meat', 'meat'),
 ('steak', 'steak'),
 ('filet mignon', 'filet mignon'),
 ('filet mignon', '8oz'),
 ('filet mignon', '10oz'),
 ('filet mignon', '12oz'),
 ('chicken', 'chicken'),
 ('drumstick', 'drumstick'),
 ('quarter', 'quarter')]

In [17]:
def expand_code(code: int) -> List[int]:
    code = str(code)
    power = len(code) -1
    expanded = list()
    for digit in code:
        if power == -1:
            break
        expanded.append(int(digit)*10**power)
        power-=1
    return expanded
        
        
    

In [18]:
expand_code(54321)

[50000, 4000, 300, 20, 1]

In [35]:
def get_last_nonzero_digit(code):
    code = str(code)
    last_nonzero_index = -1
    while code[last_nonzero_index] == '0':
        last_nonzero_index -=1
    return int(code[last_nonzero_index])

In [41]:
def get_code_parent(code: int) -> int:
    #power = -1*get_last_nonzero_digit(code)
    power = 1
    return code - (code % 10**power)

In [14]:
code = 54321
code = str(code)
power = len(code) -1
expanded = list()
for digit in code:
    if power == -1:
        break
    expanded.append(int(digit)*10**power)
    power-=1
expanded

    

[50000, 4000, 300, 20, 1]

In [49]:
get_code_parent(5400)

5400

In [51]:
5400 - (5400 % 10**get_last_nonzero_digit(5400))

0

In [54]:
get_last_nonzero_digit(5400)*10

4

In [15]:
sum(expanded)

54321

In [8]:
10**5

100000

In [26]:
code

'54321'

In [27]:
code[-1]

'1'

In [36]:
get_last_nonzero_digit(12300)

3

In [37]:
get_last_nonzero_digit(15000)

5